In [1]:
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2
from openproblems.tasks.regulatory_effect_prediction import datasets, methods



In [ ]:
adata = datasets.snare_p0_braincortex(test=True)

In [ ]:
adata = methods.rp_simple(adata, n_top_genes=2000)

In [ ]:
adata.obsm

In [ ]:
%autoreload 2

import seaborn as sns
from openproblems.tasks.regulatory_effect_prediction import metrics
cors = metrics.spearman_correlation(adata)
print(cors)

In [ ]:
%autoreload 2

from openproblems.tasks.regulatory_effect_prediction import metrics
import seaborn as sns
cors = metrics.pearson_correlation(adata)

print(cors)